# Notebook 03 - Exploratory Data Analysis (EDA)

In this notebook, we will explore the cleaned dataset that we prepared earlier in Notebook 2 (02_Data_Loading_And_Preprocessing.ipynb). The dataset contains information about patients, including their age, gender, cognitive test scores, health status, and lifestyle habits.
Our goal is to understand how these different features relate to the diagnosis of Alzheimer’s disease. We will look at distributions, compare groups, and study relationships between variables to find out which features are most useful for predicting the risk of Alzheimer’s.

This step is important because it helps us decide which data to use in the machine learning model that we will build later in Notebook 4 (04_Model_Training_and_Evaluation). It also supports our overall project goal of using Business Intelligence and AI to help doctors identify people who might be at risk earlier.

## Understanding Values in the Cleaned Dataset
The dataset we are working with has been cleaned and is now ready for analysis and modeling. All missing values have been handled, and the structure is complete. The features describe various aspects of each patient’s health, habits, and cognitive condition. Below is a breakdown of the columns grouped by category. This overview helps us understand what each variable represents and how it fits into the overall picture of Alzheimer’s risk. These categories guide us as we choose which features to analyze, scale, or model.

#### Demographics
- **PatientID** – Unique identifier for each patient (not used for prediction)  
- **Age** – Patient’s age, mostly between 60 and 90  
- **Gender** – 0 = Male, 1 = Female  
- **Ethnicity** – 0 = Caucasian, 1 = African American, 2 = Asian, 3 = Other  
- **EducationLevel** – 0 to 3, where higher numbers represent more education  

#### Lifestyle
- **Smoking** – 0 = No, 1 = Yes  
- **AlcoholConsumption** – Numeric value showing amount of alcohol consumed  
- **PhysicalActivity** – Activity level score  
- **DietQuality** – Diet health score  
- **SleepQuality** – Sleep health score  
- **BMI** – Body Mass Index  

#### Medical History
- **FamilyHistoryAlzheimers** – 0 = No, 1 = Yes (genetic link)  
- **CardiovascularDisease** – 0 = No, 1 = Yes  
- **Diabetes** – 0 = No, 1 = Yes  
- **Depression** – 0 = No, 1 = Yes  
- **HeadInjury** – 0 = No, 1 = Yes  
- **Hypertension** – 0 = No, 1 = Yes  

#### Clinical Values
- **SystolicBP** – Systolic blood pressure  
- **DiastolicBP** – Diastolic blood pressure  
- **CholesterolTotal** – Total cholesterol  
- **CholesterolLDL** – "Bad" cholesterol  
- **CholesterolHDL** – "Good" cholesterol  
- **CholesterolTriglycerides** – Another cholesterol type  

#### Cognitive and Functional Assessments
- **MMSE** – Mini-Mental State Exam score (0–30)  
- **FunctionalAssessment** – Measures daily function  
- **ADL** – Activities of Daily Living score  
- **MemoryComplaints** – 0 = No, 1 = Yes  
- **BehavioralProblems** – 0 = No, 1 = Yes  

#### Symptoms and Behavior
- **Confusion** – 0 = No, 1 = Yes  
- **Disorientation** – 0 = No, 1 = Yes  
- **PersonalityChanges** – 0 = No, 1 = Yes  
- **DifficultyCompletingTasks** – 0 = No, 1 = Yes  
- **Forgetfulness** – 0 = No, 1 = Yes  

#### Target
- **Diagnosis** – This is the target column: 0 = No Alzheimer’s, 1 = Alzheimer’s  

#### Removed Column
- **DoctorInCharge** – Removed due to containing anonymized text, not relevant for analysis  

---------------------------------

## Research Questions

In this notebook, we will look into some of the main research questions from our project, using data analysis to explore and better understand them:

**1. Can we predict the risk of Alzheimer's disease based on demographic and lifestyle factors such as age, gender, physical activity, and diet?**
- We will look for clear patterns that separate patients with Alzheimer’s from those without.
- We will check if features like age, MMSE score, or physical activity are linked to diagnosis outcomes.

**2. Which health and lifestyle features are most predictive of an Alzheimer’s diagnosis?**
- We will explore which features show strong differences between groups and are good candidates for use in a prediction model.

---------------------------

## Load Our Environment

We reuse the same setup from notebook 02. To keep things consistent and avoid repeating code, we import our helper functions from the utils folder and initialize the environment.

In [12]:
# Add the parent folder to the Python path so we can import files from the "utils" folder
import sys
sys.path.append("..")

# Import custom helper functions from our project
from utils.setup_notebook import init_environment, load_csv, quick_overview
from utils.save_tools import save_plot, save_notebook_and_summary

# Import commonly used libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# Run the setup function to create folders and apply default styles
init_environment()

Environment setup complete.


-------------------------

## Load and Preview Data

In this notebook we work on three datsets. First we load our inital raw dataset and check for outliers. 

In [17]:
# Load initial dataset
df = load_csv("../data/alzheimers_disease_raw_data.csv")

# Show first few rows
df.head()

Loaded data from ../data/alzheimers_disease_raw_data.csv with shape (2149, 35)


,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,FamilyHistoryAlzheimers,CardiovascularDisease,Diabetes,Depression,HeadInjury,Hypertension,SystolicBP,DiastolicBP,CholesterolTotal,CholesterolLDL,CholesterolHDL,CholesterolTriglycerides,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,0,0,1,1,0,0,142,72,242.366840,56.150897,33.682563,162.189143,21.463532,6.518877,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,0,0,0,0,0,0,115,64,231.162595,193.407996,79.028477,294.630909,20.613267,7.118696,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,1,0,0,0,0,0,99,116,284.181858,153.322762,69.772292,83.638324,7.356249,5.895077,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,0,0,0,0,0,0,118,115,159.582240,65.366637,68.457491,277.577358,13.991127,8.965106,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,0,0,0,0,0,0,94,117,237.602184,92.869700,56.874305,291.198780,13.517609,6.045039,0,0,0.014691,0,0,1,1,0,0,XXXConfid


--------------

## Visual Exploration
In this section, we use visualizations to understand the distribution of features, identify class imbalances, and detect outliers or patterns that may be relevant for machine learning models.

# Boxplots

A boxplot is a visual summary that shows how data is distributed for each numerical feature. It helps us quickly identify central values, spread, and outliers. 
These outliers are not necessarily errors; they might represent rare but important cases. Here is how to read the boxplots:

- The red line in the center of the box is the median — the middle value of the data.
- The blue box represents the interquartile range (IQR) — from the 25th percentile (Q1) to the 75th percentile (Q3), where most of the data lies.
- The “whiskers” are the lines extending from the box, showing the rest of the distribution within a reasonable range.
- Small black dots (or circles) outside the whiskers are called outliers — values that are significantly lower or higher than the rest of the data.